## Advent of code 2020 day 11-20
See https://adventofcode.com/

In [ ]:
# note that this notebook requires the .venv-pypy environment for pypy 3.9
# to activate it from a git bash shell: source .venv-pypy/Scripts/activate
# to generate its requirements: pip freeze > .venv-pypy-requirements.txt

import collections
import itertools
import re
import copy
import math
import sys
import time
import json
import heapq
import random
import sortedcontainers
#import cProfile

In [ ]:
# utility functions and version check

def get_line_groups(lines):
    '''return list of lists of lines, each separated by empty lines, ignores empty lines from start and end'''
    lines=list(lines)
    lines.append('') # add terminator
    res=[]
    group=[]
    for line in lines:
        line=line.strip()
        if len(line)>0:
            group.append(line)
        elif len(group)>0: # close group
            res.append(group)
            group=[]
    return res

class StopExecution(Exception):
    def _render_traceback_(self):
        pass

def exit():
    raise StopExecution()
    
print(f'python version: {sys.version}')
print(f'# start_ts={int(time.time())}') # supports ranking using an honor system, before starting include this line
# in the header of your solution (which should start with a line like # 2019 day 2), then whenever you want save
# a private leaderboard json file, and run python privaterank.py filename.json

In [ ]:
# 2020 day 11
# mv ~/Downloads/input data_src/2020-day-11-input.txt
# big input file looks like: ca. 60x120 map, mostly L
# idea: part 1 parse as grid of 0/1/2, then transform to next state, only have to compare to previous
# part 2: instead of adjc_count make firstseat_count, to speed it up we create a map that lists all 
#  coords that are 'first-visible' based on the starting map to speed it up

sample1='''
L.LL.LL.LL
LLLLLLL.LL
L.L.L..L..
LLLL.LL.LL
L.LL.LL.LL
L.LLLLL.LL
..L.L.....
LLLLLLLLLL
L.LLLLLL.L
L.LLLLL.LL
'''

sample2='''
.......#.
...#.....
.#.......
.........
..#L....#
....#....
.........
#........
...#.....
'''

sample3='''
.............
.L.L.#.#.#.#.
.............
'''

sample4='''
.##.##.
#.#.#.#
##...##
...L...
##...##
#.#.#.#
.##.##.
'''

def get_firstseat_map(data): # return a lists of lists of same form, but instead of nums
    # there are lists of (y,x) tuples of first-visible seats
    nrows=len(data)
    ncols=len(data[0])
    ndata=[]
    for cy in range(nrows):
        nrow=[]
        for cx in range(ncols):
            tups=[]
            # to the right
            for x in range(cx+1, ncols):
                if data[cy][x]!=0:
                    tups.append( (cy, x) )
                    break
            # to the left
            for x in range(cx-1, -1, -1):
                if data[cy][x]!=0:
                    tups.append( (cy, x) )
                    break
            # down
            for y in range(cy+1, nrows):
                if data[y][cx]!=0:
                    tups.append( (y, cx) )
                    break
            # up
            for y in range(cy-1, -1, -1):
                if data[y][cx]!=0:
                    tups.append( (y, cx) )
                    break
            # to the right down
            y=cy+1
            for x in range(cx+1, ncols):
                if y>=nrows:
                    break                
                if data[y][x]!=0:
                    tups.append( (y, x) )
                    break
                y+=1
            # to the right up
            y=cy-1
            for x in range(cx+1, ncols):
                if y<0:
                    break                
                if data[y][x]!=0:
                    tups.append( (y, x) )
                    break
                y-=1
            # to the left down
            y=cy+1
            for x in range(cx-1, -1, -1):
                if y>=nrows:
                    break                
                if data[y][x]!=0:
                    tups.append( (y, x) )
                    break
                y+=1
            # to the left up
            y=cy-1
            for x in range(cx-1, -1, -1):
                if y<0:
                    break                
                if data[y][x]!=0:
                    tups.append( (y, x) )
                    break
                y-=1
            nrow.append(tups)
        ndata.append(nrow)
    return ndata

def get_firstseat_count(data, y, x, smap):
    tups=smap[y][x]
    count=0
    for ty, tx in tups:
        if data[ty][tx]==2:
            count+=1
    return count

def transform(data, smap): # one-turn transformation into copy of data
    ndata=[]
    for y in range(len(data)):
        row=data[y]
        nrow=[]
        for x in range(len(row)):
            num=row[x]
            if num==0:
                nrow.append(0)
            else:
                adjc=get_firstseat_count(data, y, x, smap)
                if num==1 and adjc<1:
                    nrow.append(2)
                elif num==2 and adjc>=5:
                    nrow.append(1)
                else:
                    nrow.append(num)
        ndata.append(nrow)
    return ndata

def is_equal(data_a, data_b):
    assert len(data_a) == len(data_b)
    for y in range(len(data_a)):
        if data_a[y] != data_b[y]:
            return False
    return True

def get_counts(data):
    count=collections.Counter()
    for row in data:
        for num in row:
            count[num]+=1
    return count

sample1=open('data_src/2020-day-11-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
cmap={'.': 0, 'L': 1, '#': 2}
data0=[ [ cmap[c] for c in s] for s in lines ]
smap=get_firstseat_map(data0)
#print(smap[4][3])

data1=copy.deepcopy(data0)
while True:
    data2=transform(data1, smap)
    if is_equal(data1, data2):
        counts=get_counts(data2)
        print(f'stopped with {counts[2]} seats occupied')
        break
    data1=data2


In [ ]:
# TEMPLATE
# 2020 day 11
# start_ts=RUN FIRST CELL TO GET TIME CODE BEFORE OPENING THE ASSIGNMENT
# mv ~/Downloads/input* data_src/2020-day-11-input.txt
# big input file looks like: 
# idea: part 1 parse ..., then ...

sample2='''

'''

sample1=open('data_src/2020-day-11-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
data=[ int(s) for s in lines[0].split(',') ]
groups=get_line_groups(lines)
data0=[ s.split() for s in lines ]
data0=[ [cmd, int(num), 0] for cmd, num in data0 ]
data=[ result.group(1, 2, 3, 4, 5, 6, 7) for s in lines if (result:= re.match(r'(\w+)\s*x=([\d\-]+)\.\.([\d\-]+),y=([\d\-]+)\.\.([\d\-]+),z=([\d\-]+)\.\.([\d\-]+)', s)) ]
data=[ (row[0], int(row[1]), int(row[2]), int(row[3]), int(row[4]), int(row[5]), int(row[6]) ) for row in data ]
# template, remove what's not needed